<a href="https://colab.research.google.com/github/MeharKalra/GESTURON/blob/main/Gesturon_Experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [64]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import classification_report

In [65]:
class_dict = {"Non-gesture":0, 'Pointing with one finger':1,'Pointing with two fingers':2,'Throw up':3,'Throw down':4,
              'Throw left':5,'Throw right':6,'Open twice':7,'Zoom in':8,'Zoom out':9}

In [66]:
train_raw_path = "/train_flow_vector.csv"
train_label_path = "/Labels.csv"
test_raw_path = "/test_flow_vector.csv"
test_label_path = "/Labels (1).csv"

train_traj_path = "/temporal_diff_traj.csv"
test_traj_path = "/temporal_diff_traj_test.csv"


In [67]:
def ProcessData(df,label_df):
  for index,row in label_df.iterrows():
    Label = row["Label"]
    if Label == "Double click with one finger":
      df.drop(index,inplace=True)
      label_df.drop(index,inplace=True)
    elif Label == "Double click with two fingers":
      df.drop(index,inplace=True)
      label_df.drop(index,inplace=True)
    elif Label == "Click with one finger":
      df.drop(index,inplace=True)
      label_df.drop(index,inplace=True)
    elif Label == "Click with two fingers":
      df.drop(index,inplace=True)
      label_df.drop(index,inplace=True)

In [68]:
def ProcessDataNew(data):
  for index,row in data.iterrows():
      Label = row["label"]
      if Label == "Double click with one finger":
        data.drop(index,inplace=True)
      elif Label == "Double click with two fingers":
        data.drop(index,inplace=True)
      elif Label == "Click with one finger":
        data.drop(index,inplace=True)
      elif Label == "Click with two fingers":
        data.drop(index,inplace=True)


In [69]:
def MakeData(path_list,Raw=True,scale=False,reduced=False):
  if Raw:
    train_X = pd.read_csv(path_list[0])
    train_Y = pd.read_csv(path_list[1])
    test_X = pd.read_csv(path_list[2])
    test_Y = pd.read_csv(path_list[3])

    ProcessData(train_X,train_Y)
    ProcessData(test_X,test_Y)

  else:
    train_data = pd.read_csv(path_list[0])
    test_data = pd.read_csv(path_list[1])
    ProcessDataNew(train_data)
    ProcessDataNew(test_data)
    train_X = train_data.drop("label",axis=1)
    train_Y = pd.DataFrame(train_data["label"])
    test_X = test_data.drop("label",axis=1)
    test_Y = pd.DataFrame(test_data["label"])

  if scale:
    scaler = StandardScaler()
    train_X = scaler.fit_transform(train_X)
    test_X = scaler.transform(test_X)

  if reduced:
    pca = PCA(n_components=0.95)  # Retain 95% variance
    train_X = pca.fit_transform(train_X)
    test_X = pca.transform(test_X)

  return train_X,train_Y,test_X,test_Y


## SVM CLASSIFIER EXPERIEMNT

In [70]:
def SVM(train_X,test_X,train_Y,test_Y,C):
  train_Y = train_Y.values.ravel()
  test_Y = test_Y.values.ravel()
  print("Parmeter C: ",C)
  svm_classifier = SVC(kernel='rbf', random_state=42, C=C)
  svm_classifier.fit(train_X, train_Y)

  y_pred_train = svm_classifier.predict(train_X)
  accuracy = accuracy_score(train_Y, y_pred_train)
  print(f"Accuracy of the SVM classifier on Training set: {accuracy:.2f}")

  y_pred_test = svm_classifier.predict(test_X)
  accuracy = accuracy_score(test_Y, y_pred_test)
  print(f"Accuracy of the classifier on Testing set: {accuracy:.2f}")
  print("classification Report")
  print(classification_report(test_Y, y_pred_test,target_names=list(class_dict.keys())))

  #return y_pred_train,y_pred_test

## NEURAL NETWORK EXPERIMENT

In [71]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

In [72]:
def NeuralNetwork(train_X,test_X,train_Y,test_Y,learning_rate,n_input,reduced=False):
  encoder = OneHotEncoder(sparse_output=False)
  train_Y_onehot = encoder.fit_transform(train_Y)
  test_Y_onehot = encoder.transform(test_Y)

  if reduced:
    model = Sequential([
      Dense(128, activation='relu', input_shape=(n_input,)),  # First hidden layer
      Dropout(0.3),                                     # Dropout for regularization
      Dense(64, activation='relu'),                      # Second hidden layer
      Dropout(0.3),                                        # Dropout for regularization
      Dense(10, activation='softmax')  ])                  # Output layer for 10 classes
  else:
      model = Sequential([
      Dense(256, activation='relu', input_shape=(n_input,)),  # First hidden layer
      Dropout(0.3),                                     # Dropout for regularization
      Dense(128, activation='relu'),                      # Second hidden layer
      Dropout(0.3),                                        # Dropout for regularization
      Dense(10, activation='softmax')                     # Output layer for 10 classes
    ])

  # Compile the model with Adam optimizer and categorical crossentropy
  model.compile(optimizer=Adam(learning_rate=learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])

  # Early stopping to prevent overfitting
  early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

  # Train the model
  model.fit(train_X, train_Y_onehot, epochs=50, batch_size=32, validation_split=0.2, callbacks=[early_stopping], verbose=1)

  # Evaluate the model on the test set
  train_loss, train_accuracy = model.evaluate(train_X, train_Y_onehot, verbose=0)
  print(f"Train Accuracy: {train_accuracy * 100:.2f}%")
  test_loss, test_accuracy = model.evaluate(test_X,test_Y_onehot, verbose=0)
  print(f"Test Accuracy: {test_accuracy * 100:.2f}%")




## Experiment

In [73]:
from os import path
def make_repoert():
  #Make Raw data
  print("Without trajectory features")
  path_list = [train_raw_path,train_label_path,test_raw_path,test_label_path]
  print("Un-processed Data :")
  train_X,train_Y,test_X,test_Y = MakeData(path_list,Raw=True,scale=False,reduced=False)
  print("Number of features :",train_X.shape[1])
  print("SVM")
  C = [1,10]
  for c in C:
    SVM(train_X,test_X,train_Y,test_Y,c)

  print("Neural Network")
  learning_rate = [0.001,0.0001]
  n_input = train_X.shape[1]
  for lr in learning_rate:
    print("Learning Rate :",lr)
    NeuralNetwork(train_X,test_X,train_Y,test_Y,lr,n_input,reduced=False)

  print("Scaled Data")
  train_X,train_Y,test_X,test_Y = MakeData(path_list,Raw=True,scale=True,reduced= False)
  print("Number of features :",train_X.shape[1])

  print("SVM")
  C = [1,10]
  for c in C:
    SVM(train_X,test_X,train_Y,test_Y,c)

  print("Neural Network")
  learning_rate = [0.001,0.0001]
  n_input = train_X.shape[1]
  for lr in learning_rate:
    print("Learning Rate :",lr)
    NeuralNetwork(train_X,test_X,train_Y,test_Y,lr,n_input,reduced=False)

  print("PCA Data")
  train_X,train_Y,test_X,test_Y = MakeData(path_list,Raw=True,scale=True,reduced=True)
  print("Number of features :",train_X.shape[1])

  print("SVM")
  C = [1,10]
  for c in C:
    SVM(train_X,test_X,train_Y,test_Y,c)

  print("Neural Network")
  learning_rate = [0.001,0.0001]
  n_input = train_X.shape[1]
  for lr in learning_rate:
    print("Learning Rate :",lr)
    NeuralNetwork(train_X,test_X,train_Y,test_Y,lr,n_input,reduced=True)

  print("With trajectory features")
  path_list = [train_traj_path,test_traj_path]

  train_X,train_Y,test_X,test_Y = MakeData(path_list,Raw=False,scale=False,reduced=False)
  print("Number of features :",train_X.shape[1])
  print("SVM")
  C = [1,10]
  for c in C:
    SVM(train_X,test_X,train_Y,test_Y,c)

  print("Neural Network")
  learning_rate = [0.001,0.0001]
  n_input = train_X.shape[1]
  for lr in learning_rate:
    print("Learning Rate :",lr)
    NeuralNetwork(train_X,test_X,train_Y,test_Y,lr,n_input,reduced=False)

  print("Scaled Data")
  train_X,train_Y,test_X,test_Y = MakeData(path_list,Raw=False,scale=True,reduced= False)
  print("Number of features :",train_X.shape[1])

  print("SVM")
  C = [1,10]
  for c in C:
    SVM(train_X,test_X,train_Y,test_Y,c)

  print("Neural Network")
  learning_rate = [0.001,0.0001]
  n_input = train_X.shape[1]
  for lr in learning_rate:
    print("Learning Rate :",lr)
    NeuralNetwork(train_X,test_X,train_Y,test_Y,lr,n_input,reduced=False)

  print("PCA Data")
  train_X,train_Y,test_X,test_Y = MakeData(path_list,Raw=False,scale=True,reduced=True)
  print("Number of features :",train_X.shape[1])

  print("SVM")
  C = [1,10]
  for c in C:
    SVM(train_X,test_X,train_Y,test_Y,c)

  print("Neural Network")
  learning_rate = [0.001,0.0001]
  n_input = train_X.shape[1]
  for lr in learning_rate:
    print("Learning Rate :",lr)
    NeuralNetwork(train_X,test_X,train_Y,test_Y,lr,n_input,reduced=True)




In [74]:
make_repoert()

Without trajectory features
Un-processed Data :
Number of features : 3087
SVM
Parmeter C:  1
Accuracy of the SVM classifier on Training set: 0.80
Accuracy of the classifier on Testing set: 0.63
classification Report
                           precision    recall  f1-score   support

              Non-gesture       0.47      0.69      0.56        81
 Pointing with one finger       0.86      0.63      0.73        51
Pointing with two fingers       0.41      0.35      0.38        49
                 Throw up       0.43      0.57      0.49        51
               Throw down       0.90      0.75      0.82        51
               Throw left       0.82      0.65      0.73        51
              Throw right       0.82      0.71      0.76        52
               Open twice       0.76      0.75      0.75        51
                  Zoom in       0.67      0.66      0.67        50
                 Zoom out       0.53      0.51      0.52        49

                 accuracy                    

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 4s 33ms/step - accuracy: 0.1780 - loss: 2.2694 - val_accuracy: 0.3411 - val_loss: 2.0568
Epoch 2/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - accuracy: 0.4212 - loss: 1.9199 - val_accuracy: 0.5184 - val_loss: 1.6335
Epoch 3/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.6051 - loss: 1.4138 - val_accuracy: 0.5920 - val_loss: 1.3705
Epoch 4/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.6960 - loss: 1.1026 - val_accuracy: 0.6522 - val_loss: 1.2568
Epoch 5/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.7534 - loss: 0.8719 - val_accuracy: 0.6890 - val_loss: 1.1655
Epoch 6/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.8059 - loss: 0.7091 - val_accuracy: 0.7057 - val_loss: 1.1478
Epoch 7/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.8404 - loss: 0.5855 - val_accuracy: 0.7124 - val_loss: 1.1258
Epoch 8/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.8921 - loss: 0.4662 - val_accuracy: 0.6689 - v

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


38/38 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - accuracy: 0.0979 - loss: 2.2974 - val_accuracy: 0.1773 - val_loss: 2.2836
Epoch 2/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.2220 - loss: 2.2761 - val_accuracy: 0.2274 - val_loss: 2.2650
Epoch 3/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.2796 - loss: 2.2540 - val_accuracy: 0.2943 - val_loss: 2.2420
Epoch 4/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.3373 - loss: 2.2180 - val_accuracy: 0.3077 - val_loss: 2.2127
Epoch 5/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.3726 - loss: 2.1831 - val_accuracy: 0.3311 - val_loss: 2.1767
Epoch 6/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.3624 - loss: 2.1417 - val_accuracy: 0.3512 - val_loss: 2.1368
Epoch 7/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.4114 - loss: 2.0784 - val_accuracy: 0.3645 - val_loss: 2.0921
Epoch 8/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.4060 - loss: 2.0343 - val_accuracy: 0.4047 - val_loss: 2.

...

38/38 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - accuracy: 0.1009 - loss: 2.8929 - val_accuracy: 0.2642 - val_loss: 2.2479
Epoch 2/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.2041 - loss: 2.3188 - val_accuracy: 0.4013 - val_loss: 2.0483
Epoch 3/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.3441 - loss: 1.9916 - val_accuracy: 0.4448 - val_loss: 1.9390
Epoch 4/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.4239 - loss: 1.7757 - val_accuracy: 0.4682 - val_loss: 1.8589
Epoch 5/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.5123 - loss: 1.5596 - val_accuracy: 0.5217 - val_loss: 1.7948
Epoch 6/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.5212 - loss: 1.4938 - val_accuracy: 0.5284 - val_loss: 1.7273
Epoch 7/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.6007 - loss: 1.3265 - val_accuracy: 0.5585 - val_loss: 1.6834
Epoch 8/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 2s 23ms/step - accuracy: 0.6471 - loss: 1.2435 - val_accuracy: 0.5518 - val_loss: 1.

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


38/38 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - accuracy: 0.1092 - loss: 3.7708 - val_accuracy: 0.2308 - val_loss: 2.3400
Epoch 2/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.1932 - loss: 2.5846 - val_accuracy: 0.3278 - val_loss: 2.0829
Epoch 3/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.3299 - loss: 2.1371 - val_accuracy: 0.4381 - val_loss: 1.9379
Epoch 4/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.4183 - loss: 1.8689 - val_accuracy: 0.4849 - val_loss: 1.8295
Epoch 5/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5175 - loss: 1.6228 - val_accuracy: 0.5151 - val_loss: 1.7103
Epoch 6/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5381 - loss: 1.4839 - val_accuracy: 0.5753 - val_loss: 1.6157
Epoch 7/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5955 - loss: 1.3659 - val_accuracy: 0.5853 - val_loss: 1.5120
Epoch 8/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6681 - loss: 1.1628 - val_accuracy: 0.6020 - val_loss: 1.4401
Ep

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


38/38 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.1101 - loss: 4.2732 - val_accuracy: 0.0903 - val_loss: 3.4866
Epoch 2/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.1114 - loss: 3.9896 - val_accuracy: 0.1037 - val_loss: 3.2265
Epoch 3/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.1402 - loss: 3.8395 - val_accuracy: 0.1137 - val_loss: 3.0297
Epoch 4/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.1192 - loss: 3.6862 - val_accuracy: 0.1271 - val_loss: 2.8995
Epoch 5/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.1530 - loss: 3.3830 - val_accuracy: 0.1371 - val_loss: 2.8014
Epoch 6/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.1429 - loss: 3.2778 - val_accuracy: 0.1405 - val_loss: 2.7180
Epoch 7/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.1695 - loss: 3.0978 - val_accuracy: 0.1472 - val_loss: 2.6530
Epoch 8/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.1788 - loss: 3.0427 - val_accuracy: 0.1605 - val_loss: 2.6031
Epo